# Exercise 3: Parallel ETL

## Overview
1. Tạo Redshift cluster -> Lấy Endpoint
2. Tạo IAM role -> Lấy ARN
3. Dùng `redshift_connector` để connect Redshift cluster -> Tạo table -> Load data vào table từ S3 bucket của Udacity (có dùng IAM role ở step 2)

In [10]:
import configparser
import redshift_connector

# STEP 1: Get the params of the created redshift cluster
1. Tạo 1 Redshift cluster rồi lấy Endpoint (bỏ phần cuối `:5439/`) điền vào file `dwh.cfg`

<img src="images/aws1.png" width=1000>

2. Tạo 1 IAM role rồi lấy ARN điền vào file `dwh.cfg`

<img src="images/aws2.png" width=1000>

# STEP 2: Connect to the Redshift Cluster

In [13]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [14]:
conn = redshift_connector.connect(
    host=config['CLUSTER']['HOST'],
    database=config['CLUSTER']['DB_NAME'],
    port=config['CLUSTER']['DB_PORT'],
    user=config['CLUSTER']['DB_USER'],
    password=config['CLUSTER']['DB_PASSWORD']
)
conn.autocommit = True
cur = conn.cursor()

In [15]:
import boto3

s3 = boto3.resource(service_name='s3',
                    region_name="us-west-2",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

sampleDbBucket =  s3.Bucket("udacity-labs")

for obj in sampleDbBucket.objects.filter(Prefix="tickets"):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/')
s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/full/')
s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/full/full.csv.gz')
s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/split/')
s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/split/part-00000-d33afb94-b8af-407d-abd5-59c0ee8f5ee8-c000.csv.gz')
s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/split/part-00001-d33afb94-b8af-407d-abd5-59c0ee8f5ee8-c000.csv.gz')
s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/split/part-00002-d33afb94-b8af-407d-abd5-59c0ee8f5ee8-c000.csv.gz')
s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/split/part-00003-d33afb94-b8af-407d-abd5-59c0ee8f5ee8-c000.csv.gz')
s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/split/part-00004-d33afb94-b8af-407d-abd5-59c0ee8f5ee8-c000.csv.gz')
s3.ObjectSummary(bucket_name='udacity-labs', key='tickets/split/part-00005-d33afb94-b8af-407d-abd5-

# STEP 3: Create Tables

In [18]:
cur.execute("""
    DROP TABLE IF EXISTS "sporting_event_ticket"
""")
cur.execute("""
    CREATE TABLE "sporting_event_ticket" (
        "id" double precision DEFAULT nextval('sporting_event_ticket_seq') NOT NULL,
        "sporting_event_id" double precision NOT NULL,
        "sport_location_id" double precision NOT NULL,
        "seat_level" numeric(1,0) NOT NULL,
        "seat_section" character varying(15) NOT NULL,
        "seat_row" character varying(10) NOT NULL,
        "seat" character varying(10) NOT NULL,
        "ticketholder_id" double precision,
        "ticket_price" numeric(8,2) NOT NULL
    );
""")

# STEP 4: Load Partitioned data into the cluster

In [19]:
%%time
qry = """
    copy sporting_event_ticket from 's3://udacity-labs/tickets/split/part'
    credentials 'aws_iam_role={}'
    gzip delimiter ';' compupdate off region 'us-west-2';
""".format(DWH_ROLE_ARN)

cur.execute(qry)

CPU times: user 2.42 ms, sys: 1.82 ms, total: 4.23 ms
Wall time: 38.6 s


# STEP 5: Create Tables for the non-partitioned data

In [20]:
cur.execute("""
    DROP TABLE IF EXISTS "sporting_event_ticket_full";
""")
cur.execute("""
    CREATE TABLE "sporting_event_ticket_full" (
        "id" double precision DEFAULT nextval('sporting_event_ticket_seq') NOT NULL,
        "sporting_event_id" double precision NOT NULL,
        "sport_location_id" double precision NOT NULL,
        "seat_level" numeric(1,0) NOT NULL,
        "seat_section" character varying(15) NOT NULL,
        "seat_row" character varying(10) NOT NULL,
        "seat" character varying(10) NOT NULL,
        "ticketholder_id" double precision,
        "ticket_price" numeric(8,2) NOT NULL
    );
""")

# STEP 6: Load non-partitioned data into the cluster
Use the COPY command to load data from `s3://udacity-labs/tickets/full/full.csv.gz` using your iam role credentials. Use gzip delimiter `;`.

Note how it's slower than loading partitioned data

In [21]:
%%time

qry = """
    copy sporting_event_ticket_full from 's3://udacity-labs/tickets/full/full.csv.gz' 
    credentials 'aws_iam_role={}' 
    gzip delimiter ';' compupdate off region 'us-west-2';
""".format(DWH_ROLE_ARN)

cur.execute(qry)

CPU times: user 1.98 ms, sys: 2.69 ms, total: 4.67 ms
Wall time: 26.7 s
